In [ ]:
cd /home/ruofanl/Projects/stable-virtual-camera

/home/ruofanl/Projects/stable-virtual-camera


In [ ]:
import sys
sys.path.append('/home/ruofanl/Projects/video-to-video')
sys.path.insert(0, '/home/ruofanl/Projects/diffusers/src')
from src.models.custom_unet_mv import UNetMV2DConditionModel

In [ ]:
import glob
import os
import os.path as osp

import fire
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from tqdm import tqdm

from seva.data_io import get_parser
from seva.eval import (
    IS_TORCH_NIGHTLY,
    compute_relative_inds,
    create_transforms_simple,
    infer_prior_inds,
    infer_prior_stats,
    run_one_scene,
)
from seva.geometry import (
    generate_interpolated_path,
    generate_spiral_path,
    get_arc_horizontal_w2cs,
    get_default_intrinsics,
    get_lookat,
    get_preset_pose_fov,
)
from seva.model import SGMWrapper
from seva.modules.autoencoder import AutoEncoder
from seva.modules.conditioner import CLIPConditioner
from seva.sampling import DiscreteDenoiser
from seva.utils import load_model

In [ ]:
device = "cuda"

In [ ]:
MODEL = SGMWrapper(
    load_model(
        model_version=1.1,
        pretrained_model_name_or_path="stabilityai/stable-virtual-camera",
        weight_name="model.safetensors",
        device="cpu",
        verbose=True,
    ).eval()
) #.to(device)

In [ ]:
MODEL.to('cpu')

SGMWrapper(
  (module): Seva(
    (time_embed): Sequential(
      (0): Linear(in_features=320, out_features=1280, bias=True)
      (1): SiLU()
      (2): Linear(in_features=1280, out_features=1280, bias=True)
    )
    (input_blocks): ModuleList(
      (0): TimestepEmbedSequential(
        (0): Conv2d(11, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1-2): 2 x TimestepEmbedSequential(
        (0): ResBlock(
          (in_layers): Sequential(
            (0): GroupNorm32(32, 320, eps=1e-05, affine=True)
            (1): SiLU()
            (2): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (emb_layers): Sequential(
            (0): SiLU()
            (1): Linear(in_features=1280, out_features=320, bias=True)
          )
          (dense_emb_layers): Sequential(
            (0): Conv2d(6, 640, kernel_size=(1, 1), stride=(1, 1))
          )
          (out_layers): Sequential(
            (0): GroupNorm32(32, 320, eps=1e-0

In [ ]:
def count_parameters(model):
    """
    Calculate the total number of parameters in a torch.nn.Module.

    Args:
        model (torch.nn.Module): The model to count parameters for.

    Returns:
        int: Total number of parameters.
    """
    return sum(p.numel() for p in model.parameters())

count_parameters(MODEL)

1263968004

In [ ]:
named_params = MODEL.named_parameters()

In [ ]:
named_params = list(named_params)

In [ ]:
keys = [k for k, _ in named_params]
np.savetxt('seva_keys.txt', keys, fmt='%s')

In [ ]:
unet = UNetMV2DConditionModel.from_pretrained('/home/ruofanl/Projects/exp_outputs/SEVA/unet', subfolder="unet")

In [ ]:
seva_state_dict = MODEL.state_dict()
diff_state_dict = unet.state_dict()

In [ ]:
from state_dict_converter_simple import convert_seva_to_savediff_state_dict

converted_state_dict = convert_seva_to_savediff_state_dict(seva_state_dict)

Successfully mapped 1146 keys
Conversion complete: 1146 parameters in output state_dict


In [ ]:
set(converted_state_dict.keys()) - set(diff_state_dict.keys())

set()

In [ ]:
# check if converted_state_dict and diff_state_dict has the same tensor shape for each k-v
for k in converted_state_dict:
    if k in diff_state_dict:
        if converted_state_dict[k].shape != diff_state_dict[k].shape:
            print(f"Shape mismatch for key '{k}': converted {converted_state_dict[k].shape} vs diff {diff_state_dict[k].shape}")
    else:
        print(f"Key '{k}' not found in diff_state_dict")


In [ ]:
unet.load_state_dict(converted_state_dict)

<All keys matched successfully>

In [ ]:
# unet.save_pretrained('/home/ruofanl/Projects/exp_outputs/SEVA/unet')

In [ ]:
MODEL = SGMWrapper(
    load_model(
        model_version=1.1,
        pretrained_model_name_or_path="stabilityai/stable-virtual-camera",
        weight_name="model.safetensors",
        device="cpu",
        verbose=True,
    ).eval()
) #.to(device)

In [ ]:
unet = UNetMV2DConditionModel.from_pretrained('/home/ruofanl/Projects/exp_outputs/SEVA/unet', subfolder="unet")

In [ ]:
x = torch.randn(2, 11, 64, 64)
t = torch.ones(1)
d = torch.randn(2, 6, 64, 64)
c = torch.zeros(2, 1, 1024)

In [ ]:
with torch.no_grad():
    y = MODEL.module(x, t, c, d, num_frames=2)

proj_in: torch.Size([2, 4096, 320]) 3397.57421875
block: torch.Size([2, 4096, 320]) 85991.7109375
mix_block: torch.Size([2, 4096, 320]) 47366.40234375
proj_in: torch.Size([2, 4096, 320]) 16099.732421875
block: torch.Size([2, 4096, 320]) 39218.3984375
mix_block: torch.Size([2, 4096, 320]) 59618.75
proj_in: torch.Size([2, 1024, 640]) 12355.9306640625
block: torch.Size([2, 1024, 640]) -24538.669921875
mix_block: torch.Size([2, 1024, 640]) -16010.703125
proj_in: torch.Size([2, 1024, 640]) -4545.09375
block: torch.Size([2, 1024, 640]) 4445.7783203125
mix_block: torch.Size([2, 1024, 640]) 10369.7109375
proj_in: torch.Size([2, 256, 1280]) 7140.7333984375
block: torch.Size([2, 256, 1280]) 7564.3994140625
mix_block: torch.Size([2, 256, 1280]) 17577.921875
proj_in: torch.Size([2, 256, 1280]) 22444.798828125
block: torch.Size([2, 256, 1280]) 4174.41357421875
mix_block: torch.Size([2, 256, 1280]) 7135.63525390625
Down 0: torch.Size([2, 320, 64, 64]), 13385.8154296875
Down 1: torch.Size([2, 320, 64

In [ ]:
y.shape

torch.Size([2, 4, 64, 64])

In [ ]:
with torch.no_grad():
    yy = unet(x[None, :], t, c, d[None, :])

proj_in: torch.Size([2, 4096, 320]) 4796.1162109375
block: torch.Size([2, 4096, 320]) 91528.7890625
mix_block: torch.Size([2, 4096, 320]) 72354.671875
proj_in: torch.Size([2, 4096, 320]) 46760.72265625
block: torch.Size([2, 4096, 320]) 24770.029296875
mix_block: torch.Size([2, 4096, 320]) 8932.080078125
proj_in: torch.Size([2, 1024, 640]) -3866.392333984375
block: torch.Size([2, 1024, 640]) -53681.16796875
mix_block: torch.Size([2, 1024, 640]) -62955.3984375
proj_in: torch.Size([2, 1024, 640]) 1255.724365234375
block: torch.Size([2, 1024, 640]) 20222.91796875
mix_block: torch.Size([2, 1024, 640]) 28811.916015625
proj_in: torch.Size([2, 256, 1280]) 13911.08203125
block: torch.Size([2, 256, 1280]) 25709.4375
mix_block: torch.Size([2, 256, 1280]) 48988.2265625
proj_in: torch.Size([2, 256, 1280]) 3566.4296875
block: torch.Size([2, 256, 1280]) -16818.5390625
mix_block: torch.Size([2, 256, 1280]) -1035.089599609375
  Down block 0: torch.Size([2, 320, 64, 64]) 12496.8515625
  Down block 1: to

In [ ]:
yy.sample.shape

torch.Size([1, 2, 4, 64, 64])

In [ ]:
y[0, :, :3, :3]

tensor([[[ 0.0296, -0.2650,  0.2572],
         [-0.0848, -0.0635, -0.0595],
         [-0.0150,  0.0219,  0.1421]],

        [[-0.0394, -0.2449, -0.3784],
         [-0.0346,  0.0697,  0.3272],
         [-0.5023, -0.1836,  0.1464]],

        [[-0.1654, -0.1294, -0.2493],
         [ 0.0814,  0.0313, -0.2740],
         [-0.2658,  0.3167, -0.1194]],

        [[-0.7795,  0.1478, -0.4227],
         [-0.1096,  0.1960,  0.5011],
         [-0.2916,  0.3913, -0.1746]]])

In [ ]:
yy.sample[0, 0, :, :3, :3]

NameError: name 'yy' is not defined

In [ ]:
## Denoising scheduler
